In [2]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(reshape2))
suppressPackageStartupMessages(library(DBI))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(foreach))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))

In [1]:
# # Read the meta data
# clin_table <- "models_clinical_2020-01-14"
# sanctumdbcon <- RMariaDB::dbConnect(
#     drv = RMariaDB::MariaDB(),
#     dbname = "Devdb",
#     user = "admin",
#     password = "JohnLennon1980",
#     host = "sanctum-instance-1.cnklj0yzvfnv.us-east-1.rds.amazonaws.com",
#     port = 3306
#   )
# clin_data <- dbReadTable(sanctumdbcon, name = clin_table)
# dbDisconnect(sanctumdbcon)

In [2]:
# clin_data

In [9]:
# clin_data <- dcast(subset(clin_data, !is.na(key)), 
#                  formula = model_id ~ key,
#                  fun.aggregate = max) %>%
# subset(., Model %in% colnames(counts)) # keep metadata for relavant samples

# colnames(clin_data) <- gsub('..via.TruSight.clinical.', '', colnames(clin_data))
# clin_data[] <- lapply(clin_data, toupper)
# clin_data[clin_data=="PENDING" | clin_data=="NOT AVAILABLE" | clin_data=="OTHER" ] <- NA
# clin_data[] <- lapply(clin_data, function(x) gsub("DIFFERENTIATED", "DIFF.", x))
# clin_data[] <- lapply(clin_data, function(x) gsub("SQUAMOUS CELL", "SC", x))
# # remove (largely) empty columns 
# clin_data <- clin_data[apply(clin_data, 2, function(x) sum(is.na(x)) < nrow(clin_data)/2)]
# # identify numeric columns
# clin_data <- retype(clin_data)

### Proteomics data

In [13]:
counts  <- read.csv("./data/Prot_Data_F20FTSUSAT1137_47AML_2021-03-29.csv")
head(counts, 2)

X,ProteinGroupId,ProteinGroup,Protein,SingleProtein,Sample,Intensity,Counts,ProteinGroupName,hgnc_symbol
1,A0A075B759,sp|A0A075B759|PAL4E_HUMAN;sp|F5H284|PAL4D_HUMAN,sp|A0A075B759|PAL4E_HUMAN,sp|A0A075B759|PAL4E_HUMAN,CTG-2226,17.225,153194,PAL4E_HUMAN,PPIAL4E
2,A0A075B759,sp|A0A075B759|PAL4E_HUMAN;sp|F5H284|PAL4D_HUMAN,sp|A0A075B759|PAL4E_HUMAN,sp|A0A075B759|PAL4E_HUMAN,CTG-2227,18.070,275177,PAL4E_HUMAN,PPIAL4E


In [14]:
### Missingness, filtering and imputation ###
NAs_persamp <- sapply(Filter(is.numeric, counts), function(x) sum(is.na(x)))
# NAs_perfeature <- rowSums(is.na(Filter(is.numeric, counts)))
# prcntNAs_persamp <- round(NAs_persamp/length(NAs_perfeature)*100, 1)
# prcntNAs_perfeature <- round(NAs_perfeature/length(NAs_persamp)*100, 1)

In [15]:
NAs_persamp

X Intensity    Counts 
        0         0         0

### From Art

In [3]:
prot_intensity <- read.csv("./data/Protein_intensity.csv")
head(prot_intensity, 10)

id,Protein,ProteinGroup,ProteinGroupName,ProteinGroupId,Organism,Sample,Intensity,counts,hgnc_symbol
45243,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0158,15.503,NULL,NULL
45244,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0159,14.932,NULL,NULL
45245,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0160,14.791,NULL,NULL
45246,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0162,15.616,NULL,NULL
45247,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0163,15.860,NULL,NULL
45248,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0166,15.620,NULL,NULL
45249,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0167,14.404,NULL,NULL
45250,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0178,17.208,NULL,NULL
45251,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0184,15.961,NULL,NULL
45252,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0464,14.648,NULL,NULL


In [9]:
min(prot_intensity$Intensity)

[1] -10.92784

In [10]:
max(prot_intensity$Intensity)

[1] 28.30615

In [16]:
# unique(prot_intensity$counts)

In [4]:
phos <- read.csv("./data/Protein_phos_data.csv")
head(phos, 10)

id,ProteinGroupId,ProteinGroup,PhosLocation_Protein,SingleProtein,Sample,Intensity,Counts,ProteinGroupName,hgnc_symbol
1,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-0464,10.762,1737,TVA40_HUMAN,TRAV40
2,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-0502,12.949,7907,TVA40_HUMAN,TRAV40
3,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-0838,11.048,2117,TVA40_HUMAN,TRAV40
4,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-0852,9.974,1006,TVA40_HUMAN,TRAV40
5,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-1012,14.859,29717,TVA40_HUMAN,TRAV40
6,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-1329,10.467,1415,TVA40_HUMAN,TRAV40
7,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-1352,13.136,9002,TVA40_HUMAN,TRAV40
8,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-1355,11.033,2095,TVA40_HUMAN,TRAV40
9,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-1360,13.278,9933,TVA40_HUMAN,TRAV40
10,A0A0B4J280,sp|A0A0B4J280|TVA40_HUMAN,S85,sp|A0A0B4J280|TVA40_HUMAN,CTG-1361,14.094,17487,TVA40_HUMAN,TRAV40
